## Introduction

Jupyter notebooks have become the de facto standard for interactive computing
and data analysis, combining code, prose, and visualizations in a single
document.

This blog post was written in a notebook!

Jupyter's architecture separates the notebook interface, where users write and
interact with code (typically built with web technologies), from the **kernel**,
which executes it. This modular design has driven innovation, giving users
flexibility in both front ends (e.g., JupyterLab, VS Code) and programming
languages.

Since v1.37, Deno has included a built-in Jupyter kernel, bringing JavaScript
and TypeScript to data science and machine learning. Having worked extensively
with computational notebooks (mainly in Python), I find this exciting for
several reasons:

- **Easier setup** – The kernel is built into the Deno CLI, so there’s no need
  for additional installation. Just install Deno.

- **Improved dependency management** – Notebooks often behave like standalone
  scripts, making dependency management a challenge in other languages and
  contributing to
  [reproducibility issues](https://leomurta.github.io/papers/pimentel2019a.pdf).
  Deno's use of ECMAScript modules allows dependencies to be declared directly
  in code, improving self-containment and reliability of analysis code.

- **A unified ecosystem for interactive data analysis** – Jupyter frontends rely
  on web technologies and support rich outputs in HTML, CSS, and JavaScript.
  Since JavaScript ecosystem dominates interactive UI development, Deno bridges
  the gap between the kernel and front end, unlocking new possibilities for data
  science and machine learning.

In this post, we'll analyze the National Gallery of Art's Open Access data in
Jupyter with Deno. We'll start with basic data cleaning, manipulation, and
plotting to explore which artworks are public domain, who created them, where
they're from, and any interesting patterns. The techniques should feel familiar
to those who've worked in Jupyter before, but we'll introduce some of the key
libraries available for working with data in Deno (e.g., `npm:nodejs-polars`,
`npm:@observablehq/plot`). Finally, we'll integrate interactive widgets and
extend Jupyter with a custom display to make exploring the data more dynamic and
open-ended.

## The Dataset

The **National Gallery of Art (NGA)
[Open Data Program](https://www.nga.gov/open-access-images/open-data.html)**
provides access to over **130,000 artworks** and their creators, available
[on GitHub](https://github.com/NationalGalleryOfArt/opendata/tree/main/data).
This dataset includes valuable metadata such as titles, dates, artists, and
classifications, all under a
[Creative Commons 0 (CC0)](https://creativecommons.org/public-domain/cc0/)
license, meaning it's free to use and share.

The collection spans a wide variety of artworks, from sculptures to paintings,
by famous artists like Mary Cassatt, M.C. Escher, Vincent van Gogh, Pablo
Picasso, and Georgia O'Keeffe.

However, navigating this resource is challenging. The National Gallery's website
is not very user-friendly, making it difficult to make sense of what is actually
_in_ the dataset. It's nearly impossible to get any high-level insights, like
the number of paintings, which time periods are most represented, or which
artists are most prevalent–let alone make specific queries.

Fortunately, the dataset is available
[on GitHub](https://github.com/NationalGalleryOfArt/opendata/tree/main/data) as
a collection of related tables, exported as CSV files. For our analysis, we'll
focus on three key tables:

- **`objects.csv`** – Metadata about artworks, including titles, dates,
  materials, and classifications.
- **`constituents.csv`** – Artist details such as names, nationalities, and
  lifespans.
- **`published_images.csv`** – Links to artwork images via the NGA’s IIIF API.

We'll **clean** and **join** these tables to create a unified dataset.

**But wait!** While thesee data are free to use and contains plenty of
interesting information, the dataset only includes metadata (e.g., titles,
dates, classifications). Images of the artworks are available, but under
separate licensing. Only about half of these images are CC0-licensed, meaning
they're free to download and share.

This information is **not** in our dataset, but it’s essential for understanding
the data. We’ll gather it separately to identify which images are freely
available. Then, we can explore the collection (and maybe find a new wallpaper
for your computer!).

## Loading & cleaning the data

### Open Access Dataset

To get started, we first need to download the data. If you’ve worked with CSVs
in JavaScript, it might look like a `fetch` request followed by a parser. Here,
we’re using `jsr:@std/csv` to stream the data, which gives us an array of
objects.

In [ ]:
import * as csv from "jsr:@std/csv@1.0.5";

let baseUrl = new URL(
  "https://github.com/NationalGalleryOfArt/opendata/raw/refs/heads/main/data/",
);
let response = await fetch(new URL("objects.csv", baseUrl));

let objects = await Array.fromAsync(
  response.body
    .pipeThrough(new TextDecoderStream())
    .pipeThrough(new csv.CsvParseStream({ skipFirstRow: true })),
  (row) => ({
    objectid: +row.objectid,
    title: row.title,
    year: +row.beginyear,
    medium: row.medium,
    type: row.visualbrowserclassification,
  }),
);

objects.slice(0, 3);

If we were just plotting the data directly, this approach would work great.
However, an array of objects is not an ideal data structure for our dataset,
especially since it’s relational and requires additional cleaning and wrangling
to visualize. This is where an alternative data structure, designed for
relational queries and manipulations, becomes useful: a DataFrame. DataFrames
provide explicit and efficient APIs for handling relational data and performing
complex operations.

Let’s see how we can load the same dataset using the Polars DataFrame library:

In [ ]:
import * as pl from "npm:nodejs-polars@0.18.0";

let response = await fetch(new URL("objects.csv", baseUrl));

let objects: pl.DataFrame = pl.readCSV(await response.text(), {
  quoteChar: '"',
});

objects = objects.select(
  pl.col("objectid"),
  pl.col("title"),
  pl.col("beginyear").as("year"),
  pl.col("medium"),
  pl.col("visualbrowserclassification").as("type"),
);

objects.head();

Here, we `fetch` the dataset as before, but instead of parsing the CSV
ourselves, we read it directly with Polars to create a `pl.DataFrame`.

We then chain a `.select` expression to choose and rename the columns we want.
Notice how, rather than operating on each row in a loop, with Polars we work
directly with columns in the dataset. This is a much higher-level API, and
importantly, it allows us to perform efficient operations without materializing
JavaScript objects. Polars handles everything in Rust, letting us use high-level
APIs for processing.

Let's load our other two datasets as `pl.DataFrames` as well:

The `constituents.csv` table contains information about any person or entity
associated with a piece of art, such as artists, curators, or collectors.

In [ ]:
let response = await fetch(new URL("constituents.csv", baseUrl));
let constituents = pl.readCSV(await response.text(), { quoteChar: '"' })
  .select(
    "constituentid",
    pl.col("forwarddisplayname").alias("name"),
    pl.col("visualbrowsernationality").alias("nationality"),
  );

constituents.head();

The `published_images.csv` table contains additional information about the
artwork's image, including a URL for the thumbnail and mapping the objectid to
the IIIF (International Image Interoperability Framework) for the image.

In [ ]:
let response = await fetch(new URL("published_images.csv", baseUrl));
let publishedImages = pl.readCSV(await response.text(), { quoteChar: '"' })
  .select(
    pl.col("depictstmsobjectid").alias("objectid"),
    pl.col("uuid"),
    pl.col("iiifthumburl").alias("thumburl"),
  );
publishedImages.head();

The final table, `objects_constituents.csv`, represents a many-to-many
relationship between artworks and the people or entities associated with them.
Each artwork may have multiple people linked to it.

For our analysis, we're interested in selecting the "primary" constituent (i.e.,
the artist). We define the primary artist as the one with the highest
`displayorder` for each object.

In [ ]:
let response = await fetch(new URL("objects_constituents.csv", baseUrl));
let objectToArtist = pl.readCSV(await response.text(), { quoteChar: '"' })
  .filter(pl.col("role").eq(pl.lit("artist")))
  .sort({ by: "displayorder" })
  .groupBy("objectid")
  .first()
  .select("objectid", "constituentid");

objectToArtist.head();

This is a much more advanced usage of Polars, so let’s break down what's going
on.

It's important to note that this kind of manipulation would be very tedious to
write by hand. However, with `pl.DataFrame`, we get a high-level (and efficient)
API to express this series of operations. Let’s break it down step by step
(notice how the text description reflects the code below):

- **Filter** the table to select only rows where the role is "artist".
- **Sort** the filtered data by `displayorder` to ensure all entries are at the
  top.
- **Group** the rows **by** `objectid`, ensuring one row per artwork.
- Get the **first** in each group (i.e., artist with highest display order).
- **Select** just the `objectid` and `constituentid` for joining our tables.

### Public domain images IDs

We've loaded all the tables from the Open Access dataset, but we still don't
know which works of art have images that are also public domain. I could only
find this information on the website, but after some digging, I reverse was able
to reverse-engineer an API call to retrieve all the IDs.

We only need the IDs, not all the data, but the query takes time to load. Since
this is not an official API, I've saved the results separately.

Now, we have around 50,000 IDs to identify which images in our dataset are
public domain.

In [ ]:
// This is not an official API, so I’ve cached the results to avoid repeatedly querying the server.
//
// let response = await fetch(
//   "https://www.nga.gov/bin/ngaweb/collection-search-result/search.pageSize__100000.pageNumber__1.lastFacet__artobj_downloadable.json?artobj_downloadable=Image_download_available",
// );
// let data = await response.json();
// Deno.writeTextFileSync(
//   "public-domain-ids.txt",
//   data.results.map((object) => object.id).join("\n"),
// );

let publicDomainIds = Deno.readTextFileSync("public-domain-ids.txt")
  .split("\n")
  .map((d) => +d);

## Putting it together

Now that we have all our data cleaned and loaded, we'll perform a large join to
combine all these tables into a **single table**.

Again, this challenging to do manually, but with Polars, we can elegantly
express these complex join operations. Finally, we use a `.isIn` expression to
derive a new column that indicates whether a piece of art is public domain.

In [ ]:
let df = objects
  .join(objectToArtist, { on: "objectid" })
  .join(constituents, { on: "constituentid" })
  .join(publishedImages, { on: "objectid" })
  .select(pl.exclude("constituentid"))
  .withColumns(pl.col("objectid").isIn(publicDomainIds).alias("public"))
  .sort({ by: "public" })
  .sort({ by: "year", descending: true, nullsLast: true });

df.head();

## Interactive tables

In [ ]:
import { agGrid, quak } from "jsr:@manzt/jupyter-helper";

In [ ]:
agGrid(df.head(100));

In [ ]:
// quak can handle it all (keeps as compressed parquet in the front-end)
quak(df.select(pl.exclude("objectid", "uuid", "thumburl")));

## Plotting

- explain deps observable/plot
- plot some different views / EDA

In [ ]:
import * as Plot from "npm:@observablehq/plot";
import { document } from "jsr:@manzt/jupyter-helper";

let records = df.toRecords();

// Take away:
// -> Most of the data comes from print, drawings, photographs.
// -> we can see that the majority of art in cateogry is public domain, expect for photographs and portfolios

Plot.plot({
  color: { legend: true },
  marks: [
    Plot.barY(
      records,
      Plot.groupX(
        { y: "count" },
        {
          x: "type",
          sort: { x: "-y" },
          fill: (d) => d.public ? "Public domain" : "Copyrighted",
        },
      ),
    ),
  ],
  marginLeft: 125,
  width: 1000,
  document,
});

In [ ]:
// Take away: Most artwork is American and vary with regard to public domain
Plot.plot({
  color: { legend: true },
  marks: [
    Plot.barX(
      records,
      Plot.groupY(
        { x: "count" },
        {
          y: "nationality",
          sort: { y: "-x" },
          fill: (d) => d.public ? "Public domain" : "Copyrighted",
        },
      ),
    ),
  ],
  marginLeft: 125,
  document,
});

In [ ]:
// take away (most of individual's art are either public domain or not)
Plot.plot({
  color: { legend: true },
  marks: [
    Plot.barX(
      df
        .groupBy("name", "public")
        .len()
        .sort("name_count", true)
        .head(25)
        .toRecords(),
      {
        x: "name_count",
        y: "name",
        sort: { y: "-x" },
        fill: (d) => d.public ? "Public domain" : "Copyrighted",
      },
    ),
  ],
  marginLeft: 200,
  document,
});

In [ ]:
// Compare paintings and photographs (we can see there is only one photographer ("Eadweard Muybridge") with some art work that is public domain
let arts = df
  .groupBy("name", "type", "public")
  .len()
  .select(
    pl.col("name"),
    pl.col("type"),
    pl.col("public"),
    pl.col("name_count").alias("count"),
  )
  .sort({ by: "count", descending: true })
  .filter(
    pl.col("type").eq(pl.lit("photograph")),
  )
  .head(30);

let p = Plot.plot({
  color: { legend: true },
  marks: [
    Plot.barX(arts.toRecords(), {
      x: "count",
      y: "name",
      fill: (d) => d.public ? "public domain" : "not public domain",
      sort: { y: "x", reverse: true },
    }),
  ],
  marginLeft: 250,
  document,
});

let foo = "bar";
render(<h1>Hello, {foo}</h1>);

In [ ]:
let arts = df
  .groupBy("name", "type", "public")
  .len()
  .select(
    pl.col("name"),
    pl.col("type"),
    pl.col("public"),
    pl.col("name_count").alias("count"),
  )
  .sort({ by: "count", descending: true })
  .head(60);

Plot.plot({
  color: { legend: true },
  marks: [
    Plot.barX(arts.toRecords(), {
      x: "count",
      y: "name",
      fill: "type",
      sort: { y: "x", reverse: true },
      opacity: (d) => d.public ? 1 : 0.5,
    }),
  ],
  marginLeft: 180,
  document,
});

In [ ]:
let counts = df
  .groupBy("name", "type")
  .len()
  .select(
    pl.col("name"),
    pl.col("type"),
    pl.col("name_count").alias("count"),
  )
  .sort({ by: "count", descending: true });

let groups = pl.concat(
  ["drawing", "print", "photograph", "painting"].map((name) =>
    counts.filter(pl.col("type").eq(pl.lit(name))).head(20)
  ),
);

In [ ]:
// Take away: All publid dommain data is prior to 1950, there was a large dump of drawing for public domain ~1950
Plot.plot({
  y: { grid: true },
  color: { legend: true },
  marks: [
    Plot.rectY(
      df
        .filter(pl.col("year").gt(1_400))
        .toRecords(),
      Plot.binX({ y: "count" }, {
        x: (d) => new Date(d.year, 0, 1),
        fill: "type",
        fy: (d) => d.public ? "public domain" : "not public domain",
      }),
    ),
    Plot.ruleY([0]),
  ],
  marginLeft: 100,
  marginRight: 100,
  width: 1200,
  height: 400,
  document,
});

In [ ]:
let drawings = df
  .filter("public")
  .filter(pl.col("year").gt(pl.lit(1920)))
  .filter(pl.col("year").lt(pl.lit(1960)))
  .sort({ by: "year" });

Plot.plot({
  y: { grid: true },
  color: { legend: true },
  marks: [
    Plot.rectY(
      drawings.toRecords(),
      Plot.binX({ y: "count" }, {
        x: (d) => new Date(d.year, 0, 1),
        fill: "name",
      }),
    ),
    Plot.ruleY([0]),
  ],
  marginLeft: 100,
  width: 1200,
  height: 400,
  document,
});

In [ ]:
// Awesome finding...
// https://www.nga.gov/features/exhibitions/outliers-and-american-vanguard-artist-biographies/index-of-american-design.html
// render(<Gallery objects={drawings.sample(100)} />)

In [ ]:
Plot.plot({
  color: { legend: true },
  marks: [
    Plot.waffleY(
      df.sort({ by: pl.col("public").not() }).toRecords(),
      Plot.groupZ({ y: "count" }, {
        fx: "type",
        fill: "public",
        unit: 300,
        sort: { fx: "-y" },
      }),
    ),
    Plot.ruleY([0]),
  ],
  width: 1000,
  document,
});

In [ ]:
import * as React from "npm:react";
import { render } from "jsr:@manzt/jupyter-helper";

function Gallery({ objects, size = 100 }) {
  return (
    <div
      style={{
        display: "grid",
        gridTemplateColumns: `repeat(auto-fill, minmax(${size}px, 1fr))`,
        gap: "4px",
      }}
    >
      {objects
        .select("objectid", "thumburl", "title", "public")
        .map(([objectid, thumburl, title, publicDomain]) => (
          <div
            key={objectid}
            style={{ position: "relative", textAlign: "center" }}
          >
            <a
              href={`https://www.nga.gov/collection/art-object-page.${objectid}.html`}
              style={{
                display: "block",
                width: `${size}px`,
                height: `${size}px`,
                position: "relative",
              }}
            >
              <img
                src={thumburl}
                alt={title}
                style={{
                  width: "100%",
                  height: "100%",
                  objectFit: "cover",
                  borderRadius: "5px",
                }}
              />
              {publicDomain && (
                <img
                  src="https://mirrors.creativecommons.org/presskit/icons/zero.svg"
                  alt="Public Domain"
                  style={{
                    position: "absolute",
                    bottom: "3px",
                    right: "3px",
                    width: "20px",
                    height: "20px",
                    opacity: 0.60,
                  }}
                />
              )}
            </a>
          </div>
        ))}
    </div>
  );
}

render(<Gallery objects={df.sample(100)} />);

In [ ]:
render(
  <Gallery objects={df.filter(pl.col("name").eq(pl.lit("Winslow Homer")))} />,
);

In [ ]:
render(
  <Gallery
    objects={df.filter(pl.col("name").eq(pl.lit("Eadweard Muybridge"))).sample(
      100,
    )}
  />,
);